In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [2]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Kafirun/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Kafirun/metadata_109.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,2_1.wav,1,109,Al-Kafirun
1,2_2.wav,1,109,Al-Kafirun
2,2_3.wav,1,109,Al-Kafirun
3,2_4.wav,1,109,Al-Kafirun
4,2_5.wav,1,109,Al-Kafirun
5,2_6.wav,1,109,Al-Kafirun
6,1_1.wav,3,109,Al-Kafirun
7,1_2.wav,3,109,Al-Kafirun
8,1_3.wav,3,109,Al-Kafirun
9,1_4.wav,3,109,Al-Kafirun


Feature Extraction

In [3]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [4]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [5]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-6.7903435e-05 -2.9996561e-04 -5.4918649e-04 ... -2.8173813e-01
 -1.7683174e-01 -1.4564484e-01] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.16377497 -0.09723695 -0.0948813  ...  0.1389994   0.2175383
  0.02969604] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.07424396 -0.03151486 -0.06980361 ...  0.0203908   0.01740677
 -0.01700218] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[

In [6]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (6, 704, 40)
Shape of labels for fold 1: (6,)


In [7]:
mfcc_data_by_fold[3]["data"].shape

(6, 704, 40)

In [8]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [9]:
mfcc_data_by_fold[1]["data"].shape

(6, 704, 40, 1)

After Training Work, Deep Features 

In [10]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 78ms/step


In [11]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [12]:
intermediate_layer_outputs.shape

(3, 6, 1024)

In [13]:
intermediate_layer_outputs

array([[[-0.99926794, -0.9996378 ,  0.99571306, ...,  0.9994123 ,
         -0.99348074, -0.95093656],
        [-0.91919744, -0.99998575,  0.17609772, ...,  0.98502016,
          0.01055716, -0.9794611 ],
        [-0.9997844 ,  0.60875374,  0.6638551 , ...,  0.9993905 ,
         -0.9393903 , -0.9919078 ],
        [-0.9485117 , -0.99985045, -0.8305377 , ...,  0.99699485,
         -0.7283614 , -0.7042768 ],
        [-0.99995625, -0.99980193,  0.99822223, ...,  0.99865246,
          0.7477426 , -0.9970952 ],
        [-0.94492954, -0.9999467 ,  0.9999949 , ...,  0.9903205 ,
         -0.9988101 , -0.9999982 ]],

       [[-0.996519  , -1.        ,  0.99846596, ...,  0.9995114 ,
         -0.99356604, -0.9993289 ],
        [-0.9963192 , -0.7580684 ,  0.9996269 , ...,  0.9999295 ,
         -0.99869937, -0.99996   ],
        [-0.99896157, -0.8822517 ,  0.99441457, ...,  0.999981  ,
         -0.9995709 , -0.99966884],
        [-0.9976949 , -0.999955  ,  0.99947715, ...,  0.99976116,
         -0.99

In [14]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [15]:
al_kafirun_mean = mean_across_folds

In [16]:
al_kafirun_mean.shape

(6, 1024)

In [17]:
al_kafirun_mean

array([[-0.99859565, -0.9998793 ,  0.9980597 , ...,  0.9996353 ,
        -0.99564487, -0.98342174],
       [-0.9718389 , -0.9193514 ,  0.7252415 , ...,  0.99361354,
        -0.64755183, -0.9931404 ],
       [-0.9995819 ,  0.23995374,  0.88608986, ...,  0.9997847 ,
        -0.97736126, -0.9971896 ],
       [-0.9820486 , -0.34426412,  0.38964   , ...,  0.99820924,
        -0.90837574, -0.90111226],
       [-0.99892443,  0.3324742 ,  0.9993108 , ...,  0.99954337,
        -0.3986148 , -0.9988082 ],
       [-0.9538308 ,  0.25197735,  0.9999916 , ...,  0.99600005,
        -0.99208754, -0.9999955 ]], dtype=float32)

In [18]:
# Specify the file path where you want to save the .npy file
file_path = 'al_kafirun_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_kafirun_mean)

Testing of Single Audio

In [19]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/kafirun_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00038831 -0.00379187 -0.00809563 ... -0.2967401  -0.33462748
 -0.3310508 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.14582482  0.19665067  0.14216582 ... -0.01939761 -0.0354126
 -0.06722739] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.36427784 -0.3876012  -0.35537657 ... -0.3284515  -0.15879528
  0.0546159 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.10914305 -0.15099788 -0.23393716 

In [20]:
mfcc_extracted_features_single

array([[[[-4.2688257e+02],
         [ 1.3379279e+02],
         [ 3.0568695e+01],
         ...,
         [-6.6331887e-01],
         [ 1.1613737e+00],
         [ 6.0511608e+00]],

        [[-4.0300415e+02],
         [ 1.4264975e+02],
         [ 2.9030685e+01],
         ...,
         [-8.0538106e-01],
         [ 1.2683309e+00],
         [ 6.1317797e+00]],

        [[-4.0279874e+02],
         [ 1.4221011e+02],
         [ 2.7276890e+01],
         ...,
         [-3.9330444e+00],
         [ 1.1828182e+00],
         [ 8.0229206e+00]],

        ...,

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
 

In [21]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 135ms/step


In [22]:
intermediate_output

array([[-0.99999964,  0.99992454,  1.        , ...,  0.99999267,
        -0.9999693 , -0.99975157]], dtype=float32)

In [23]:
intermediate_output.shape

(1, 1024)

In [24]:
intermediate_layer_outputs

array([[[-0.99926794, -0.9996378 ,  0.99571306, ...,  0.9994123 ,
         -0.99348074, -0.95093656],
        [-0.91919744, -0.99998575,  0.17609772, ...,  0.98502016,
          0.01055716, -0.9794611 ],
        [-0.9997844 ,  0.60875374,  0.6638551 , ...,  0.9993905 ,
         -0.9393903 , -0.9919078 ],
        [-0.9485117 , -0.99985045, -0.8305377 , ...,  0.99699485,
         -0.7283614 , -0.7042768 ],
        [-0.99995625, -0.99980193,  0.99822223, ...,  0.99865246,
          0.7477426 , -0.9970952 ],
        [-0.94492954, -0.9999467 ,  0.9999949 , ...,  0.9903205 ,
         -0.9988101 , -0.9999982 ]],

       [[-0.996519  , -1.        ,  0.99846596, ...,  0.9995114 ,
         -0.99356604, -0.9993289 ],
        [-0.9963192 , -0.7580684 ,  0.9996269 , ...,  0.9999295 ,
         -0.99869937, -0.99996   ],
        [-0.99896157, -0.8822517 ,  0.99441457, ...,  0.999981  ,
         -0.9995709 , -0.99966884],
        [-0.9976949 , -0.999955  ,  0.99947715, ...,  0.99976116,
         -0.99

Euclidian Distance with Mean

In [25]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_kafirun_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_kafirun_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_kafirun_mean: {closest_index+1}")
print(f"Closest value in al_kafirun_mean: {closest_value}")

Index of closest value in al_kafirun_mean: 5
Closest value in al_kafirun_mean: [-0.99892443  0.3324742   0.9993108  ...  0.99954337 -0.3986148
 -0.9988082 ]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")